# PCA

In [2]:
# IMPORTS
import sys
sys.path.append("../")
import DatasetCreator
import torch
import os.path
from os import path
import csv
from scipy import io as sio
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler

# CONFIGS
os.environ["CUDA_VISIBLE_DEVICES"]="1"
datasetNames = ["IP","Houston","UP","Trento"]
NUM_BANDS = 30

def PCA(X , num_components):
    #Step-1
    X_meaned = X - np.mean(X , axis = 0)
     
    #Step-2
    cov_mat = np.cov(X_meaned.T)
     
    #Step-3
    eigen_values , eigen_vectors = np.linalg.eig(cov_mat)
    eigen_vectors = eigen_vectors.T
    #Step-4
    sorted_index = np.argsort(eigen_values)
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
     
# #     #Step-5
# #     eigenvector_subset = sorted_eigenvectors[:,0:num_components]
     
# #     #Step-6
# #     X_reduced = np.dot(eigenvector_subset.transpose() , X_meaned.transpose() ).transpose()
     
    return sorted_index

def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(newX,num_components=numComponents)
    return pca

for datasetName in datasetNames:
    HSI_Tr,HSI_Te,_,_ = DatasetCreator.CreateDataset(datasetName)
    HSI = np.concatenate([HSI_Tr,HSI_Te],axis=0)
    HSI = HSI.astype(np.float64)
    print("Data shape =",HSI.shape)
    shapeor =HSI.shape
    HSI = minmax_scale(HSI.reshape(HSI.shape[0] * HSI.shape[1], HSI.shape[2])).reshape((shapeor[0], shapeor[1], shapeor[2]))
    
    bands = applyPCA(HSI,numComponents =  NUM_BANDS)
    print("Bands for ",datasetName, " = ",  bands)
    with open(datasetName+"_Bands_PCA_"+str(NUM_BANDS), 'w') as x_file:
        x_file.write(str(list(bands[::-1][:NUM_BANDS])))
    with open(datasetName+"_Bands_PCA_REV_"+str(NUM_BANDS), 'w') as x_file:
        x_file.write(str(list(bands[:NUM_BANDS])))

X shape =  (145, 145, 200) 
Y shape =  (145, 145)
X1 shape after IMAGE CUBES =  (145, 145, 200) 
Y shape after IMAGE CUBES =  (145, 145)
X shape =  (145, 145, 200) 
Y shape =  (145, 145)
X1 shape after IMAGE CUBES =  (145, 145, 200) 
Y shape after IMAGE CUBES =  (145, 145)
Data shape = (290, 145, 200)
Bands for  IP  =  [ 10  12  11  23  22  29  28  27  40  33  32  42  41  44  47  50  54  57
  64  63  56  55  71  70  67  66  74  75  80  78  77  81  84  83  86  85
  96  91  90  99  88  87 101 100 104 106 112 115 114 116 118 117 120 121
 125 124 123 127 130 132 134 136 135 138 139 142 145 144 146 147 149 150
 153 152 155 157 159 158 168 167 166 161 160 169 170 176 177 178 180 182
 183 184 185 186 187 189 190 194 192 197 199 198 195 191 193 196 188 181
 179 175 174 173 172 171 164 163 165 162 156 154 151 148 143 141 140 137
 133 131 129 128 126 122 119 113 111 108 107 110 109 105  98  97 103 102
  95  94  93  92  89  82  79  76  73  72  59  58  69  68  65  61  60  62
  53  52  51  49  48  

# SNMF

In [28]:
# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import nimfa
from sklearn.metrics import accuracy_score
import sys
sys.path.append("../")
import DatasetCreator
from sklearn.decomposition import PCA
import os.path
from os import path
import csv
from scipy import io as sio
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import minmax_scale

# CONFIGS
os.environ["CUDA_VISIBLE_DEVICES"]="1"
datasetNames = ["IP"]
NUM_BANDS = 30

class BandSelection_SNMF(object):
    def __init__(self, n_band):
        self.n_band = n_band

    def predict(self, X):
        """
        :param X: with shape (n_pixel, n_band)
        :return:
        """
        # # Note that X has to reshape to (n_fea., n_sample)
        # XX = X.transpose()  # (n_band, n_pixel)
        # snmf = nimfa.Snmf(X, seed="random_c", rank=self.n_band)  # remain para. default
        snmf = nimfa.Snmf(X, rank=self.n_band, max_iter=20, version='r', eta=1.,
                          beta=1e-4, i_conv=10, w_min_change=0)
        snmf_fit = snmf()
        W = snmf.basis()  # shape: n_band * k
        H = snmf.coef()  # shape: k * n_pixel

        #  get clustering res.
        H = np.asarray(H)
        indx_sort = np.argsort(H, axis=0)  # ascend order
        
        cluster_res = indx_sort[-1].reshape(-1)
        print(np.unique(cluster_res))
        #  select band
        selected_band = []
        for c in np.unique(cluster_res):
            idx = np.nonzero(cluster_res == c)
            center = np.mean(X[:, idx[0]], axis=1).reshape((-1, 1))
            distance = np.linalg.norm(X[:, idx[0]] - center, axis=0)
            band_ = X[:, idx[0]][:, distance.argmin()]
            selected_band.append(band_)
        while selected_band.__len__() < self.n_band:
            selected_band.append(np.zeros(X.shape[0]))
        bands = np.asarray(selected_band).transpose()
        return bands
    

for datasetName in datasetNames:
    snmf = BandSelection_SNMF(NUM_BANDS)
    HSI,_ = DatasetCreator.loadData("HSITest",datasetName)
    HSI = HSI.astype(np.float64)
    print("Data shape =",HSI.shape)
#     shapeor =HSI.shape
#     HSI = minmax_scale(HSI.reshape(HSI.shape[0] * HSI.shape[1], HSI.shape[2])).reshape((shapeor[0], shapeor[1], shapeor[2]))
    data = snmf.predict(HSI.reshape(-1,HSI.shape[2]))
    print("Bands for ",datasetName, " = ",  np.unique(data))
    

Data shape = (145, 145, 200)


KeyboardInterrupt: 

# SpaBS

In [3]:
# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import nimfa
from sklearn.metrics import accuracy_score
import sys
sys.path.append("../")
import DatasetCreator
from sklearn.decomposition import PCA
import os.path
from os import path
import csv
from scipy import io as sio
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import orthogonal_mp_gram
from sklearn.preprocessing import minmax_scale

# CONFIGS
os.environ["CUDA_VISIBLE_DEVICES"]="1"
datasetNames = ["IP","Houston","UP","Trento"]
NUM_BANDS = 30


class ApproximateKSVD(object):
    def __init__(self, n_components, max_iter=10, tol=1e-6,
                 transform_n_nonzero_coefs=None):
        """
        Parameters
        ----------
        n_components:
            Number of dictionary elements
        max_iter:
            Maximum number of iterations
        tol:
            tolerance for error
        transform_n_nonzero_coefs:
            Number of nonzero coefficients to target
        """
        self.components_ = None
        self.max_iter = max_iter
        self.tol = tol
        self.n_components = n_components
        self.transform_n_nonzero_coefs = transform_n_nonzero_coefs

    def _update_dict(self, X, D, gamma):
        for j in range(self.n_components):
            I = gamma[:, j] > 0
            if np.sum(I) == 0:
                continue

            D[j, :] = 0
            g = gamma[I, j].T
            r = X[I, :] - gamma[I, :].dot(D)
            d = r.T.dot(g)
            d /= np.linalg.norm(d)
            g = r.dot(d)
            D[j, :] = d
            gamma[I, j] = g.T
        return D, gamma

    def _initialize(self, X):
        if min(X.shape) <= self.n_components:
            D = np.random.randn(self.n_components, X.shape[1])
        else:
            u, s, vt = sp.sparse.linalg.svds(X, k=self.n_components)
            D = np.dot(np.diag(s), vt)
        D /= np.linalg.norm(D, axis=1)[:, np.newaxis]
        return D

    def _transform(self, D, X):
        gram = D.dot(D.T)
        Xy = D.dot(X.T)

        n_nonzero_coefs = self.transform_n_nonzero_coefs
        if n_nonzero_coefs is None:
            n_nonzero_coefs = int(0.1 * X.shape[1])

        return orthogonal_mp_gram(
            gram, Xy, n_nonzero_coefs=n_nonzero_coefs).T

    def fit(self, X):
        """
        Parameters
        ----------
        X: shape = [n_samples, n_features]
        """
        D = self._initialize(X)
        for i in range(self.max_iter):
            gamma = self._transform(D, X)
            e = np.linalg.norm(X - gamma.dot(D))
            if e < self.tol:
                break
            D, gamma = self._update_dict(X, D, gamma)

        self.components_ = D
        return self

    def transform(self, X):
        return self._transform(self.components_, X)


class SpaBS(object):

    def __init__(self, n_band, sparsity_level=0.5):
        self.n_band = n_band
        self.sparsity_level = sparsity_level

    def fit(self, X):
        self.X = X
        return self

    def predict(self, X):
        """
        Select band according to sparse representation
        :param X: array like: shape (n_row*n_column, n_band)
        :return:
        """
        
        dico = ApproximateKSVD(n_components=X.shape[1])
        dico.fit(X)
        gamma_ = dico.transform(X) 
        gamma = gamma_.transpose()
        sorted_inx = np.argsort(gamma, axis=0)  # ascending order for each column
        
        K = X.shape[0] * self.sparsity_level
        largest_k = sorted_inx[-self.n_band:, :]

        # # statistic
        element, freq = np.unique(largest_k, return_counts=True)
        selected_inx = element[np.argsort(freq)][-self.n_band:]
        
        selected_band = X[:, selected_inx]
        return element[np.argsort(freq)]

for datasetName in datasetNames:
    spabs = SpaBS(NUM_BANDS)
    HSI_Tr,HSI_Te,_,_ = DatasetCreator.CreateDataset(datasetName)
    HSI = np.concatenate([HSI_Tr,HSI_Te],axis=0)
    HSI = HSI.astype(np.float64)
    print("Data shape =",HSI.shape)
    shapeor =HSI.shape
    HSI = minmax_scale(HSI.reshape(HSI.shape[0] * HSI.shape[1], HSI.shape[2])).reshape((shapeor[0], shapeor[1], shapeor[2]))
    HSI = HSI.reshape(-1,HSI.shape[2])
    bands = spabs.predict(HSI)
    print("Bands for ",datasetName, " = ",  bands)
    with open(datasetName+"_Bands_SpaBS_"+str(NUM_BANDS), 'w') as x_file:
        x_file.write(str(list(bands[::-1][:NUM_BANDS])))
    with open(datasetName+"_Bands_SpaBS_REV_"+str(NUM_BANDS), 'w') as x_file:
        x_file.write(str(list(bands[:NUM_BANDS])))

Data shape = (145, 145, 200)
Bands for  IP  =  [141 171 146 161 112 136 188  48 122 113  39 142  15 195 107 173  99 181
 127 139 123  18  49 185 140 155  80 119 100 143 176  41  45 126 101 128
  10 121  34  46  50  47 194 138  33  81 186 157  38 175 124  21  37 167
 147 154 179 144 130 105  85 145 169 163  86 148   9 137 150  40  42 177
  44 118 156 133 170 160  14 158 168 129 187 116 151 172  25 152  79 110
 198  29  19 104 103 111 115 192 184 134  26 108  23  36  51  97 174  22
 109   0  43 180 178 166  35 106  30 102  98  52  82 191   7 114 117  90
 193  31  94 153 189  24 165 120  20  16 149  77 197  13  95   5  88 164
  11 190  92  78  91 132  32  28 135  89   4  12  76 159  83 162  75 196
  27  96 182  53  93   6  74  87   3   1   8 131   2  54  17 125  72  73
  55  71  56  59 199  70  69  57  58  61  62  68  64  63  60  66  67  65
  84]
Data shape = (349, 1905, 144)


/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/sklearn/utils/validation.py:63: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return f(*args, **kwargs)
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/sklearn/utils/validation.py:63: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return f(*args, **kwargs)
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/sklearn/utils/validation.py:63: RuntimeWarning:  Orthogonal matching pursuit ended prematurely due to linear
dependence in the dictionary. The requested precision might not have been met.

  return f(*args, **kwargs)
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/sklearn/utils/validation.py:63: RuntimeWarning:  Orthogonal matching 

Bands for  Houston  =  [ 23  31 108 122 131  16 128  88  33 117  14  83  73  34 109  24  79  19
  72  95 105 123  86 137 119  27  85  26 103  29 134 100 107  96 125  94
   4 102  89 139 104  98  93 141 112 106  25 115 110  82 135  15 142 121
  13  20   9  28  76  80  18 138  92 101  69  70 116  97 129 133 118 111
  35   6 120  32  81   8 124  99  22 130 132  12 126 114  10   7  78  90
  71  75  74  77  62  36   5  17   0  84 127  11  67  64  60  91  61  63
  30  66  87  21 140 113  59   3   1  58  37  68  38 143  39   2  57  56
  55  54  53  40  52  51  50  41  65  45  42  43  46  44  49  48  47]
Data shape = (610, 340, 103)
Bands for  UP  =  [ 91  55  94  21  86  67  78  81  93  22  59  74  71  18  97  68  62  11
  88  52  65  53  92  57  89  58  64  95  83  84  54  15 100  80  87  16
  82  70  90   9  60  66  63  73  12 101  77  85   4  56   6  99  13  10
  75  76  17  50  19  79  69  20  23  51  61   0   7   2  14  98   8   1
  72   3  49  24  48  96  25 102  26  47  29  27  41  46 

# BS-Net-Conv

In [ ]:
# IMPORTS
import skimage
import kornia
from skimage import measure
import sys
sys.path.append("../")
import DatasetCreator
from os import path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as dataf
import scipy
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

# CONFIGS
os.environ["CUDA_VISIBLE_DEVICES"]="1"
datasetNames = ["IP","Houston","UP","Trento"]
NUM_BANDS = 30
BATCH_SIZE = 32
EPOCHS = 100

class BSNET_Conv(nn.Module):

    def __init__(self,):
      
        super(BSNET_Conv, self).__init__()
        self.conv1 = nn.Sequential(
        	nn.Conv2d(200,64,(3,3),1,0),
        	nn.ReLU(True))

        self.conv1_1 = nn.Sequential(
        	nn.Conv2d(200,128,(3,3),1,0),
        	nn.ReLU(True))
        self.conv1_2 = nn.Sequential(
        	nn.Conv2d(128,64,(3,3),1,0),
        	nn.ReLU(True))
        
        self.deconv1_2 = nn.Sequential(
        	nn.ConvTranspose2d(64,64,(3,3),1,0),
        	nn.ReLU(True))
        
        self.deconv1_1 = nn.Sequential(
        	nn.ConvTranspose2d(64,128,(3,3),1,0),
        	nn.ReLU(True))

        self.conv2_1 = nn.Sequential(
        	nn.Conv2d(128,200,(1,1),1,0),
        	nn.Sigmoid())
        

        self.fc1 = nn.Sequential(
        	nn.Linear(64,128),
        	nn.ReLU(True))
        self.fc2 = nn.Sequential(
        	nn.Linear(128,200),
        	nn.Sigmoid())
        self.gp=nn.AvgPool2d(5)
    
        
    def BAM(self,x):
        
        x = self.conv1(x)
        
        
        x = self.gp(x)
        
        
        x = x.view(-1,64)
        
        x = self.fc1(x)
        
        x = self.fc2(x)
        
        x = x.view(-1,1,1,200)
        x = x.permute(0,3,1,2)
        
        return x

    def RecNet(self,x):

      x = self.conv1_1(x)
      
      x = self.conv1_2(x)
      
      x = self.deconv1_2(x)
      
      x = self.deconv1_1(x)
      
      x = self.conv2_1(x)
      
      return x
      
      

    def forward(self,x):
      
      BRW = self.BAM(x)
      
      x = x*BRW
      ret = self.RecNet(x)
      
      return ret

def train(epoch, maxBands, optimizer):    
    model.train()
    ENTROPY = torch.zeros(maxBands)

    for batch_idx, (data, __) in enumerate(loader):
        data = data.cuda()
        optimizer.zero_grad()
        output =  model(data)
        loss = F.l1_loss(output,data)
        loss.backward()
        optimizer.step()
        D = output.detach().cpu().numpy()
        for i in range(0,maxBands):

          ENTROPY[i]+=skimage.measure.shannon_entropy(D[:,i,:,:])
        
        if batch_idx % (0.5*len(loader)) == 0:



            L1 = loss.item()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(loader.dataset),
                100. * batch_idx / len(loader),L1))
            l1_list.append(L1)
            ssim_val = torch.mean(ssim(data,output))
            print("SSIM: {}".format(ssim_val))
            ssim_list.append(ssim_val)
            psnr_val = psnr(data,output)
            print("PSNR: {}".format(psnr_val))
            psnr_list.append(psnr_val)
        
        
    ENTROPY = np.array(ENTROPY)
    bsnlist = np.asarray(ENTROPY.argsort()[-NUM_BANDS:][::-1])
    print('Top {} bands with Entropy ->'.format(NUM_BANDS),list(bsnlist))
    return bsnlist
for datasetName in datasetNames:
    ssim_list = []
    psnr_list = []
    l1_list = []
    channel_weight_list = []
    ssim = kornia.losses.SSIM(5, 1.0, 1e-12)
    psnr = kornia.losses.PSNRLoss(2500)
    HSI_Tr,HSI_Te,Labels_Tr,Labels_Te = DatasetCreator.CreateDataset(name = datasetName,windowSize = 11)
    HSI = np.concatenate([HSI_Tr,HSI_Te],axis = 0)
    Labels = np.concatenate([Labels_Tr,Labels_Te],axis = 0)
    HSI = HSI.astype(np.float32)
    Labels = Labels.astype(np.float32)
    
    HSI = torch.from_numpy(HSI)
    HSI = HSI.permute(0,3,1,2)
    Labels = torch.from_numpy(Labels)
    print(Labels.shape)
    print(HSI.shape)
    dataset = dataf.TensorDataset(HSI, Labels)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    model = BSNET_Conv().cuda()
    summary(model,(HSI.shape[1],11,11))
    finalBands = []
    bestSSIM = 0.
    optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.9)
    for epoch in range(EPOCHS):
        bands = train(epoch, HSI.shape[1],optimizer )
        finalBands = bands
   
    
    
    print("Bands for ",datasetName, " = ",  finalBands)
    with open(datasetName+"_Bands_BS-Net-Conv_"+str(NUM_BANDS), 'w') as x_file:
        x_file.write(str(finalBands))

X shape =  (145, 145, 200) 
Y shape =  (145, 145)
(155, 155, 200)
X1 shape after IMAGE CUBES =  (10249, 11, 11, 200) 
Y shape after IMAGE CUBES =  (10249,)
X shape =  (145, 145, 200) 
Y shape =  (145, 145)
(155, 155, 200)
X1 shape after IMAGE CUBES =  (10249, 11, 11, 200) 
Y shape after IMAGE CUBES =  (10249,)
torch.Size([20498])
torch.Size([20498, 200, 11, 11])
------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 9, 9]            --
|    └─Conv2d: 2-1                       [-1, 64, 9, 9]            115,264
|    └─ReLU: 2-2                         [-1, 64, 9, 9]            --
├─AvgPool2d: 1-2                         [-1, 64, 1, 1]            --
├─Sequential: 1-3                        [-1, 128]                 --
|    └─Linear: 2-3                       [-1, 128]                 8,320
|    └─ReLU: 2-4                         

Top 30 bands with Entropy -> [53, 98, 159, 89, 121, 116, 72, 88, 71, 47, 73, 173, 6, 97, 84, 148, 170, 101, 43, 115, 174, 123, 114, 163, 38, 9, 57, 82, 137, 182]
Train Epoch: 23 [0/20498 (0%)]	Loss: 0.000916
SSIM: nan
PSNR: 113.90988159179688
Top 30 bands with Entropy -> [53, 98, 101, 121, 168, 142, 89, 159, 123, 67, 137, 174, 72, 131, 100, 71, 88, 37, 116, 152, 87, 162, 84, 183, 30, 114, 173, 82, 115, 79]
Train Epoch: 24 [0/20498 (0%)]	Loss: 0.000885
SSIM: nan
PSNR: 108.77063751220703
Top 30 bands with Entropy -> [159, 98, 53, 89, 121, 116, 71, 72, 87, 88, 131, 142, 37, 67, 100, 174, 101, 84, 137, 162, 123, 183, 152, 168, 82, 114, 79, 30, 97, 57]
Train Epoch: 25 [0/20498 (0%)]	Loss: 0.000745
SSIM: nan
PSNR: 115.21627807617188
Top 30 bands with Entropy -> [53, 98, 159, 89, 116, 121, 72, 88, 71, 174, 101, 153, 11, 3, 2, 83, 51, 109, 111, 142, 167, 100, 9, 57, 68, 44, 112, 67, 143, 178]
Train Epoch: 26 [0/20498 (0%)]	Loss: 0.000681
SSIM: nan
PSNR: 114.96717834472656
Top 30 bands with Ent

In [5]:
!tf_upgrade_v2 --infile untitled.py --outfile untitled2.py

2021-11-13 10:33:54.491517: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
INFO line 127:8: Renamed 'tf.reset_default_graph' to 'tf.compat.v1.reset_default_graph'
INFO line 128:8: Renamed 'tf.set_random_seed' to 'tf.compat.v1.set_random_seed'
INFO line 132:21: Renamed 'tf.layers.batch_normalization' to 'tf.compat.v1.layers.batch_normalization'
INFO line 135:21: Renamed 'tf.layers.conv2d' to 'tf.compat.v1.layers.conv2d'
INFO line 136:57: Changing tf.contrib.layers xavier initializer to a tf.compat.v1.keras.initializers.VarianceScaling and converting arguments.

INFO line 137:30: Renamed 'tf.layers.batch_normalization' to 'tf.compat.v1.layers.batch_normalization'
INFO line 143:22: Added keywords to args of function 'tf.reduce_mean'
INFO line 144:21: Renamed 'tf.layers.dense' to 'tf.compat.v1.layers.dense'
INFO line 145:56: Changing tf.contrib.layers xavier initializer to a tf.compat.v1.keras.initializers.VarianceScali